In [3]:
# Assignment 3 Data Engineering 5hp
# Part B
# By Ture Hassler

# Initialize spark session and spark context
from pyspark.sql import SparkSession


ss = SparkSession \
    .builder \
    .master("spark://192.168.2.122:7077") \
    .appName("turehassler_partA") \
    .config("spark.dynamicAllocation.enabled", True)\
    .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
    .config("spark.shuffle.service.enabled", True)\
    .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
    .config("spark.cores.max", 16)\
    .getOrCreate()

sc = ss.sparkContext

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/02/23 17:07:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# Load data into DataFrame and show that it works
df = ss.read.csv("hdfs://host-192-168-2-122:9000/parking-citations.csv", sep=',', inferSchema=True, header=True)
df.show(3)

+-------------+-------------------+----------+--------+-----------+--------------+-----------------+----+----+----------+-----+---------------+-----+------+--------------+---------------------+-----------+---------+---------+------------------+-----------------+----------------------+
|Ticket number|         Issue Date|Issue time|Meter Id|Marked Time|RP State Plate|Plate Expiry Date| VIN|Make|Body Style|Color|       Location|Route|Agency|Violation code|Violation Description|Fine amount| Latitude|Longitude|Agency Description|Color Description|Body Style Description|
+-------------+-------------------+----------+--------+-----------+--------------+-----------------+----+----+----------+-----+---------------+-----+------+--------------+---------------------+-----------+---------+---------+------------------+-----------------+----------------------+
|   1103341116|2015-12-21 00:00:00|    1251.0|    null|       null|            CA|         200304.0|null|HOND|        PA|   GY|13147 WELBY WAY

In [5]:
print("Schema: ", df.schema)
print("Nr rows: ", df.count())
print("Nr partitions: ", df.rdd.getNumPartitions())

Schema:  StructType(List(StructField(Ticket number,StringType,true),StructField(Issue Date,TimestampType,true),StructField(Issue time,DoubleType,true),StructField(Meter Id,StringType,true),StructField(Marked Time,DoubleType,true),StructField(RP State Plate,StringType,true),StructField(Plate Expiry Date,DoubleType,true),StructField(VIN,StringType,true),StructField(Make,StringType,true),StructField(Body Style,StringType,true),StructField(Color,StringType,true),StructField(Location,StringType,true),StructField(Route,StringType,true),StructField(Agency,DoubleType,true),StructField(Violation code,StringType,true),StructField(Violation Description,StringType,true),StructField(Fine amount,DoubleType,true),StructField(Latitude,DoubleType,true),StructField(Longitude,DoubleType,true),StructField(Agency Description,StringType,true),StructField(Color Description,StringType,true),StructField(Body Style Description,StringType,true)))


Nr rows:  13077724
Nr partitions:  16


In [6]:
# Drop columns
d2 = df.drop("VIN", "Latitude", "Longitude")

In [8]:
from pyspark.sql.types import DoubleType
d3 = d2.withColumn("Fine amount", d2["Fine amount"].cast(DoubleType()))

from pyspark.sql.functions import max
d3.select(max(d3["Fine amount"])).show()


+----------------+
|max(Fine amount)|
+----------------+
|          1100.0|
+----------------+



In [13]:
# Groups by car brand, counts and displays descending order first 20 rows
d3.groupBy('Make').count().orderBy("count", ascending=False).show()

+----+-------+
|Make|  count|
+----+-------+
|TOYT|2150768|
|HOND|1479996|
|FORD|1116235|
|NISS| 945133|
|CHEV| 892676|
| BMW| 603092|
|MERZ| 543298|
|VOLK| 432030|
|HYUN| 404917|
|DODG| 391686|
|LEXS| 368420|
| KIA| 328155|
|JEEP| 316300|
|AUDI| 255395|
|MAZD| 242344|
|OTHR| 205546|
| GMC| 184889|
|INFI| 174315|
|CHRY| 159948|
|SUBA| 154640|
+----+-------+
only showing top 20 rows



In [30]:
from pyspark.sql.functions import *

COLORS = {
'AL':'Aluminum', 'AM':'Amber', 'BG':'Beige', 'BK':'Black', 'BL':'Blue', 'BN':'Brown', 'BR':'Brown', 'BZ':'Bronze', 'CH':'Charcoal', 'DK':'Dark', 'GD':'Gold', 'GO':'Gold', 'GN':'Green', 'GY':'Gray', 'GT':'Granite', 'IV':'Ivory', 'LT':'Light', 'OL':'Olive', 'OR':'Orange', 'MR':'Maroon', 'PK':'Pink', 'RD':'Red', 'RE':'Red', 'SI':'Silver', 'SL':'Silver', 'SM':'Smoke', 'TN':'Tan', 'VT':'Violet', 'WT':'White', 'WH':'White', 'YL':'Yellow', 'YE':'Yellow', 'UN':'Unknown'
}

# Actual function
def colorLong(s):
    if s in COLORS:
        return COLORS[s]
    else:
        return s

# User defined function boilerplate code
udf_colorLong = udf(lambda x: colorLong(x))

# Use withColumn to create a new column with long color names
d4 = d3.withColumn("color long", udf_colorLong(col("color")))



In [29]:
# Filter to get only Toyotoa cars
toyota_df = d4.filter(col("make") == "TOYT")

# Group the dataframe by color and count the number of occurrences of each color
color_counts = toyota_df.groupBy("color long").count().orderBy("count", ascending=False).show()

# Seems to rougly line up with the total amount of toyota cars so should be correct 

+----------+------+
|color long| count|
+----------+------+
|      Gray|489697|
|     White|434595|
|     Black|353812|
|    Silver|347894|
|      Blue|180091|
|       Red|119074|
|     Green| 74968|
|      Gold| 40646|
|    Maroon| 26242|
|       Tan| 23355|
|     Beige| 15723|
|        OT| 15719|
|     Brown| 11454|
|    Yellow|  4372|
|        PR|  4272|
|    Orange|  3575|
|   Unknown|  2012|
|        TU|  1647|
|      null|   771|
|        CO|   730|
+----------+------+
only showing top 20 rows

